In [ ]:
# ! pip install python_hiccup

In [ ]:
# ! pip install edn_format

In [ ]:
from functools import partial

def walk(inner, outer, coll):
    if isinstance(coll, list):
        return outer([inner(e) for e in coll])
    elif isinstance(coll, dict):
        return outer(dict([inner(e) for e in iter(coll.items())]))
    elif isinstance(coll, tuple):
        return outer([inner(e) for e in coll])
    else:
        return outer(coll)

def identity(e):
    return e
    
def prewalk(fn, coll):
    return walk(partial(prewalk, fn), identity, fn(coll))

def postwalk(fn, coll):
    return walk(partial(postwalk, fn), fn, coll)

def subs(s, m):
    return s[m:]

In [ ]:
import edn_format

def ednTypesToPy(edn):
    if isinstance(edn, edn_format.immutable_list.ImmutableList):
        return list(edn)
    elif isinstance(edn, edn_format.immutable_dict.ImmutableDict):
        return dict(edn)
    elif isinstance(edn, edn_format.edn_lex.Keyword):
        return subs(str(edn),1)
    else:
        return edn

def execute(edn):
    if isinstance(edn, tuple):
        return globals()[edn[0]](edn[1])
    else:
        return edn

In [ ]:
from IPython.core.magic import (Magics, magics_class, cell_magic)
from IPython.display import display, HTML
from python_hiccup.html import render

@magics_class
class MyMagics(Magics):
    @cell_magic
    def cmagic(self, line, cell):
        display(HTML(render(prewalk(ednTypesToPy,edn_format.loads(cell)))))
        return None

In [ ]:
from IPython import get_ipython
get_ipython().register_magics(MyMagics)

In [ ]:
%%cmagic
[:h1 "Hello world!"]

In [ ]:
%%cmagic
[:svg {:width 200 :height 100} [:circle {:cx 50 :cy 50 :r 25 :style "fill:red"}]]

In [ ]:
%%html
<svg width="200" height="100" xmlns="http://www.w3.org/2000/svg">
  <circle id="circle2" cx="50" cy="50" r="25" style="fill:red;" />
  Sorry, your browser does not support inline SVG.
</svg>

In [ ]:
locals()["subs"]("hi",1)

In [ ]:
def ssu(e):
    return subs(*e)

In [ ]:
l=("ssu", ["hi", 1])

In [ ]:
execute(l)

In [ ]:
locals()[l[0]](l[1])

In [ ]:
l[0]

In [ ]:
ssu(["hi",1])

In [ ]:
isinstance(l,tuple)